# Limited information goodness-of-fit tests for ordinal factor models

[Haziq Jamil](https://haziqj.ml) [](https://orcid.org/0000-0003-3298-1010) (King Abdullah University of Science and Technology, Universiti Brunei Darussalam)

Limited information approaches overcome sparsity issues and computational challenges in traditional goodness-of-fit tests. This paper describes the implementation of LIGOF tests for ordinal factor models that have been fitted using the `{lavaan}` package in R. The tests are computationally efficient and reliable, and adapted to suit whichever parameter estimation procedure was used to fit the model. The implementation is available as an R package called `{lavaan.ligof}`.

``` r
# LaTeX shortcuts 
cat(readr::read_file("_extensions/maths_shortcuts.tex"))
```

## 1 Introduction

## 2 Methods

In [ ]:
library(tidyverse)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.2     ✔ tibble    3.3.0
✔ lubridate 1.9.4     ✔ tidyr     1.3.1
✔ purrr     1.0.4     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


Attaching package: 'kableExtra'

The following object is masked from 'package:dplyr':

    group_rows

### 2.1 Ordinal data

Consider the case of analysing multivariate data $\mathbf y = (y_{1}, \ldots, y_{p})^\top$, where each item $y_{i}$ is an ordinal random variable with $m_i$ categories, $i=1,\dots,p$. Let $\mathcal R = \{ \mathbf c = (c_1,\dots, c_p)^\top \mid c_i \in \{1,\dots, m_i\}\}$ be the set of all possible response patterns, and let $R=\prod_{i} m_i$ be the cardinality of this set. The joint probability of observing a response pattern $\mathbf c_r \in \mathcal R$ is given by $$
\pi_r = \Pr(\mathbf y = \mathbf c_r) = \Pr(y_1 = \mathbf c_{r1}, \ldots, y_p = \mathbf c_{rp}), \hspace{2em} r = 1, \ldots, R,
$$ with $\sum_r \pi_R = 1$. Collect all response probabilities into the vector $\boldsymbol \pi = (\pi_1, \ldots, \pi_R)^\top \in [0,1]^R$. An example with $p=3$, $m_1=2$, and $m_2=m_3=3$ is given below. In total, there are $R=2 \times 3 \times 3 = 18$ response patterns as shown in <a href="#tbl-response-patterns" class="quarto-xref">Table 1</a>.

In [ ]:
tab_rp <-
  expand_grid(
    y1 = 1:2,
    y2 = 1:3,
    y3 = 1:3
  ) |> 
  unite("pattern", everything(), sep = "", remove = FALSE) |>
  mutate(r = row_number()) |>
  select(r, starts_with("y"), pattern)

``` r
tab_rp |>
  slice(1:9) |>
  gt() |>
  cols_label(
    r = md("$r$"),
    y1 = md("$y_1$"),
    y2 = md("$y_2$"),
    y3 = md("$y_3$"),
    pattern = "Pattern"
  ) |>
  tab_options(table.width = "80%")
tab_rp |>
  slice(-(1:9)) |>
  gt() |>
  cols_label(
    r = md("$r$"),
    y1 = md("$y_1$"),
    y2 = md("$y_2$"),
    y3 = md("$y_3$"),
    pattern = "Pattern"
  ) |>
  tab_options(table.width = "80%")
```

<table>
<colgroup>
<col style="width: 50%" />
<col style="width: 50%" />
</colgroup>
<tbody>
<tr>
<td style="text-align: left;"><div class="cell" width="50.0%" data-layout-align="left">
<div class="cell-output-display">
<div id="vsokmdluas" style="padding-left:0px;padding-right:0px;padding-top:10px;padding-bottom:10px;overflow-x:auto;overflow-y:auto;width:auto;height:auto;">
<style>#vsokmdluas table {
  font-family: system-ui, 'Segoe UI', Roboto, Helvetica, Arial, sans-serif, 'Apple Color Emoji', 'Segoe UI Emoji', 'Segoe UI Symbol', 'Noto Color Emoji';
  -webkit-font-smoothing: antialiased;
  -moz-osx-font-smoothing: grayscale;
}
&#10;#vsokmdluas thead, #vsokmdluas tbody, #vsokmdluas tfoot, #vsokmdluas tr, #vsokmdluas td, #vsokmdluas th {
  border-style: none;
}
&#10;#vsokmdluas p {
  margin: 0;
  padding: 0;
}
&#10;#vsokmdluas .gt_table {
  display: table;
  border-collapse: collapse;
  line-height: normal;
  margin-left: auto;
  margin-right: auto;
  color: #333333;
  font-size: 16px;
  font-weight: normal;
  font-style: normal;
  background-color: #FFFFFF;
  width: 80%;
  border-top-style: solid;
  border-top-width: 2px;
  border-top-color: #A8A8A8;
  border-right-style: none;
  border-right-width: 2px;
  border-right-color: #D3D3D3;
  border-bottom-style: solid;
  border-bottom-width: 2px;
  border-bottom-color: #A8A8A8;
  border-left-style: none;
  border-left-width: 2px;
  border-left-color: #D3D3D3;
}
&#10;#vsokmdluas .gt_caption {
  padding-top: 4px;
  padding-bottom: 4px;
}
&#10;#vsokmdluas .gt_title {
  color: #333333;
  font-size: 125%;
  font-weight: initial;
  padding-top: 4px;
  padding-bottom: 4px;
  padding-left: 5px;
  padding-right: 5px;
  border-bottom-color: #FFFFFF;
  border-bottom-width: 0;
}
&#10;#vsokmdluas .gt_subtitle {
  color: #333333;
  font-size: 85%;
  font-weight: initial;
  padding-top: 3px;
  padding-bottom: 5px;
  padding-left: 5px;
  padding-right: 5px;
  border-top-color: #FFFFFF;
  border-top-width: 0;
}
&#10;#vsokmdluas .gt_heading {
  background-color: #FFFFFF;
  text-align: center;
  border-bottom-color: #FFFFFF;
  border-left-style: none;
  border-left-width: 1px;
  border-left-color: #D3D3D3;
  border-right-style: none;
  border-right-width: 1px;
  border-right-color: #D3D3D3;
}
&#10;#vsokmdluas .gt_bottom_border {
  border-bottom-style: solid;
  border-bottom-width: 2px;
  border-bottom-color: #D3D3D3;
}
&#10;#vsokmdluas .gt_col_headings {
  border-top-style: solid;
  border-top-width: 2px;
  border-top-color: #D3D3D3;
  border-bottom-style: solid;
  border-bottom-width: 2px;
  border-bottom-color: #D3D3D3;
  border-left-style: none;
  border-left-width: 1px;
  border-left-color: #D3D3D3;
  border-right-style: none;
  border-right-width: 1px;
  border-right-color: #D3D3D3;
}
&#10;#vsokmdluas .gt_col_heading {
  color: #333333;
  background-color: #FFFFFF;
  font-size: 100%;
  font-weight: normal;
  text-transform: inherit;
  border-left-style: none;
  border-left-width: 1px;
  border-left-color: #D3D3D3;
  border-right-style: none;
  border-right-width: 1px;
  border-right-color: #D3D3D3;
  vertical-align: bottom;
  padding-top: 5px;
  padding-bottom: 6px;
  padding-left: 5px;
  padding-right: 5px;
  overflow-x: hidden;
}
&#10;#vsokmdluas .gt_column_spanner_outer {
  color: #333333;
  background-color: #FFFFFF;
  font-size: 100%;
  font-weight: normal;
  text-transform: inherit;
  padding-top: 0;
  padding-bottom: 0;
  padding-left: 4px;
  padding-right: 4px;
}
&#10;#vsokmdluas .gt_column_spanner_outer:first-child {
  padding-left: 0;
}
&#10;#vsokmdluas .gt_column_spanner_outer:last-child {
  padding-right: 0;
}
&#10;#vsokmdluas .gt_column_spanner {
  border-bottom-style: solid;
  border-bottom-width: 2px;
  border-bottom-color: #D3D3D3;
  vertical-align: bottom;
  padding-top: 5px;
  padding-bottom: 5px;
  overflow-x: hidden;
  display: inline-block;
  width: 100%;
}
&#10;#vsokmdluas .gt_spanner_row {
  border-bottom-style: hidden;
}
&#10;#vsokmdluas .gt_group_heading {
  padding-top: 8px;
  padding-bottom: 8px;
  padding-left: 5px;
  padding-right: 5px;
  color: #333333;
  background-color: #FFFFFF;
  font-size: 100%;
  font-weight: initial;
  text-transform: inherit;
  border-top-style: solid;
  border-top-width: 2px;
  border-top-color: #D3D3D3;
  border-bottom-style: solid;
  border-bottom-width: 2px;
  border-bottom-color: #D3D3D3;
  border-left-style: none;
  border-left-width: 1px;
  border-left-color: #D3D3D3;
  border-right-style: none;
  border-right-width: 1px;
  border-right-color: #D3D3D3;
  vertical-align: middle;
  text-align: left;
}
&#10;#vsokmdluas .gt_empty_group_heading {
  padding: 0.5px;
  color: #333333;
  background-color: #FFFFFF;
  font-size: 100%;
  font-weight: initial;
  border-top-style: solid;
  border-top-width: 2px;
  border-top-color: #D3D3D3;
  border-bottom-style: solid;
  border-bottom-width: 2px;
  border-bottom-color: #D3D3D3;
  vertical-align: middle;
}
&#10;#vsokmdluas .gt_from_md > :first-child {
  margin-top: 0;
}
&#10;#vsokmdluas .gt_from_md > :last-child {
  margin-bottom: 0;
}
&#10;#vsokmdluas .gt_row {
  padding-top: 8px;
  padding-bottom: 8px;
  padding-left: 5px;
  padding-right: 5px;
  margin: 10px;
  border-top-style: solid;
  border-top-width: 1px;
  border-top-color: #D3D3D3;
  border-left-style: none;
  border-left-width: 1px;
  border-left-color: #D3D3D3;
  border-right-style: none;
  border-right-width: 1px;
  border-right-color: #D3D3D3;
  vertical-align: middle;
  overflow-x: hidden;
}
&#10;#vsokmdluas .gt_stub {
  color: #333333;
  background-color: #FFFFFF;
  font-size: 100%;
  font-weight: initial;
  text-transform: inherit;
  border-right-style: solid;
  border-right-width: 2px;
  border-right-color: #D3D3D3;
  padding-left: 5px;
  padding-right: 5px;
}
&#10;#vsokmdluas .gt_stub_row_group {
  color: #333333;
  background-color: #FFFFFF;
  font-size: 100%;
  font-weight: initial;
  text-transform: inherit;
  border-right-style: solid;
  border-right-width: 2px;
  border-right-color: #D3D3D3;
  padding-left: 5px;
  padding-right: 5px;
  vertical-align: top;
}
&#10;#vsokmdluas .gt_row_group_first td {
  border-top-width: 2px;
}
&#10;#vsokmdluas .gt_row_group_first th {
  border-top-width: 2px;
}
&#10;#vsokmdluas .gt_summary_row {
  color: #333333;
  background-color: #FFFFFF;
  text-transform: inherit;
  padding-top: 8px;
  padding-bottom: 8px;
  padding-left: 5px;
  padding-right: 5px;
}
&#10;#vsokmdluas .gt_first_summary_row {
  border-top-style: solid;
  border-top-color: #D3D3D3;
}
&#10;#vsokmdluas .gt_first_summary_row.thick {
  border-top-width: 2px;
}
&#10;#vsokmdluas .gt_last_summary_row {
  padding-top: 8px;
  padding-bottom: 8px;
  padding-left: 5px;
  padding-right: 5px;
  border-bottom-style: solid;
  border-bottom-width: 2px;
  border-bottom-color: #D3D3D3;
}
&#10;#vsokmdluas .gt_grand_summary_row {
  color: #333333;
  background-color: #FFFFFF;
  text-transform: inherit;
  padding-top: 8px;
  padding-bottom: 8px;
  padding-left: 5px;
  padding-right: 5px;
}
&#10;#vsokmdluas .gt_first_grand_summary_row {
  padding-top: 8px;
  padding-bottom: 8px;
  padding-left: 5px;
  padding-right: 5px;
  border-top-style: double;
  border-top-width: 6px;
  border-top-color: #D3D3D3;
}
&#10;#vsokmdluas .gt_last_grand_summary_row_top {
  padding-top: 8px;
  padding-bottom: 8px;
  padding-left: 5px;
  padding-right: 5px;
  border-bottom-style: double;
  border-bottom-width: 6px;
  border-bottom-color: #D3D3D3;
}
&#10;#vsokmdluas .gt_striped {
  background-color: rgba(128, 128, 128, 0.05);
}
&#10;#vsokmdluas .gt_table_body {
  border-top-style: solid;
  border-top-width: 2px;
  border-top-color: #D3D3D3;
  border-bottom-style: solid;
  border-bottom-width: 2px;
  border-bottom-color: #D3D3D3;
}
&#10;#vsokmdluas .gt_footnotes {
  color: #333333;
  background-color: #FFFFFF;
  border-bottom-style: none;
  border-bottom-width: 2px;
  border-bottom-color: #D3D3D3;
  border-left-style: none;
  border-left-width: 2px;
  border-left-color: #D3D3D3;
  border-right-style: none;
  border-right-width: 2px;
  border-right-color: #D3D3D3;
}
&#10;#vsokmdluas .gt_footnote {
  margin: 0px;
  font-size: 90%;
  padding-top: 4px;
  padding-bottom: 4px;
  padding-left: 5px;
  padding-right: 5px;
}
&#10;#vsokmdluas .gt_sourcenotes {
  color: #333333;
  background-color: #FFFFFF;
  border-bottom-style: none;
  border-bottom-width: 2px;
  border-bottom-color: #D3D3D3;
  border-left-style: none;
  border-left-width: 2px;
  border-left-color: #D3D3D3;
  border-right-style: none;
  border-right-width: 2px;
  border-right-color: #D3D3D3;
}
&#10;#vsokmdluas .gt_sourcenote {
  font-size: 90%;
  padding-top: 4px;
  padding-bottom: 4px;
  padding-left: 5px;
  padding-right: 5px;
}
&#10;#vsokmdluas .gt_left {
  text-align: left;
}
&#10;#vsokmdluas .gt_center {
  text-align: center;
}
&#10;#vsokmdluas .gt_right {
  text-align: right;
  font-variant-numeric: tabular-nums;
}
&#10;#vsokmdluas .gt_font_normal {
  font-weight: normal;
}
&#10;#vsokmdluas .gt_font_bold {
  font-weight: bold;
}
&#10;#vsokmdluas .gt_font_italic {
  font-style: italic;
}
&#10;#vsokmdluas .gt_super {
  font-size: 65%;
}
&#10;#vsokmdluas .gt_footnote_marks {
  font-size: 75%;
  vertical-align: 0.4em;
  position: initial;
}
&#10;#vsokmdluas .gt_asterisk {
  font-size: 100%;
  vertical-align: 0;
}
&#10;#vsokmdluas .gt_indent_1 {
  text-indent: 5px;
}
&#10;#vsokmdluas .gt_indent_2 {
  text-indent: 10px;
}
&#10;#vsokmdluas .gt_indent_3 {
  text-indent: 15px;
}
&#10;#vsokmdluas .gt_indent_4 {
  text-indent: 20px;
}
&#10;#vsokmdluas .gt_indent_5 {
  text-indent: 25px;
}
&#10;#vsokmdluas .katex-display {
  display: inline-flex !important;
  margin-bottom: 0.75em !important;
}
&#10;#vsokmdluas div.Reactable > div.rt-table > div.rt-thead > div.rt-tr.rt-tr-group-header > div.rt-th-group:after {
  height: 0px !important;
}
</style>
<table class="gt_table" data-quarto-disable-processing="false" data-quarto-bootstrap="false">
  <thead>
    <tr class="gt_col_headings">
      <th class="gt_col_heading gt_columns_bottom_border gt_right" rowspan="1" colspan="1" scope="col" id="r"><span data-qmd-base64="JHIk"><span class='gt_from_md'>\(r\)</span></span></th>
      <th class="gt_col_heading gt_columns_bottom_border gt_right" rowspan="1" colspan="1" scope="col" id="y1"><span data-qmd-base64="JHlfMSQ="><span class='gt_from_md'>\(y_1\)</span></span></th>
      <th class="gt_col_heading gt_columns_bottom_border gt_right" rowspan="1" colspan="1" scope="col" id="y2"><span data-qmd-base64="JHlfMiQ="><span class='gt_from_md'>\(y_2\)</span></span></th>
      <th class="gt_col_heading gt_columns_bottom_border gt_right" rowspan="1" colspan="1" scope="col" id="y3"><span data-qmd-base64="JHlfMyQ="><span class='gt_from_md'>\(y_3\)</span></span></th>
      <th class="gt_col_heading gt_columns_bottom_border gt_right" rowspan="1" colspan="1" scope="col" id="pattern">Pattern</th>
    </tr>
  </thead>
  <tbody class="gt_table_body">
    <tr><td headers="r" class="gt_row gt_right">1</td>
<td headers="y1" class="gt_row gt_right">1</td>
<td headers="y2" class="gt_row gt_right">1</td>
<td headers="y3" class="gt_row gt_right">1</td>
<td headers="pattern" class="gt_row gt_right">111</td></tr>
    <tr><td headers="r" class="gt_row gt_right">2</td>
<td headers="y1" class="gt_row gt_right">1</td>
<td headers="y2" class="gt_row gt_right">1</td>
<td headers="y3" class="gt_row gt_right">2</td>
<td headers="pattern" class="gt_row gt_right">112</td></tr>
    <tr><td headers="r" class="gt_row gt_right">3</td>
<td headers="y1" class="gt_row gt_right">1</td>
<td headers="y2" class="gt_row gt_right">1</td>
<td headers="y3" class="gt_row gt_right">3</td>
<td headers="pattern" class="gt_row gt_right">113</td></tr>
    <tr><td headers="r" class="gt_row gt_right">4</td>
<td headers="y1" class="gt_row gt_right">1</td>
<td headers="y2" class="gt_row gt_right">2</td>
<td headers="y3" class="gt_row gt_right">1</td>
<td headers="pattern" class="gt_row gt_right">121</td></tr>
    <tr><td headers="r" class="gt_row gt_right">5</td>
<td headers="y1" class="gt_row gt_right">1</td>
<td headers="y2" class="gt_row gt_right">2</td>
<td headers="y3" class="gt_row gt_right">2</td>
<td headers="pattern" class="gt_row gt_right">122</td></tr>
    <tr><td headers="r" class="gt_row gt_right">6</td>
<td headers="y1" class="gt_row gt_right">1</td>
<td headers="y2" class="gt_row gt_right">2</td>
<td headers="y3" class="gt_row gt_right">3</td>
<td headers="pattern" class="gt_row gt_right">123</td></tr>
    <tr><td headers="r" class="gt_row gt_right">7</td>
<td headers="y1" class="gt_row gt_right">1</td>
<td headers="y2" class="gt_row gt_right">3</td>
<td headers="y3" class="gt_row gt_right">1</td>
<td headers="pattern" class="gt_row gt_right">131</td></tr>
    <tr><td headers="r" class="gt_row gt_right">8</td>
<td headers="y1" class="gt_row gt_right">1</td>
<td headers="y2" class="gt_row gt_right">3</td>
<td headers="y3" class="gt_row gt_right">2</td>
<td headers="pattern" class="gt_row gt_right">132</td></tr>
    <tr><td headers="r" class="gt_row gt_right">9</td>
<td headers="y1" class="gt_row gt_right">1</td>
<td headers="y2" class="gt_row gt_right">3</td>
<td headers="y3" class="gt_row gt_right">3</td>
<td headers="pattern" class="gt_row gt_right">133</td></tr>
  </tbody>
  &#10;  
</table>
</div>
</div>
</div></td>
<td style="text-align: left;"><div class="cell" width="50.0%" data-layout-align="left">
<div class="cell-output-display">
<div id="nogavjqqce" style="padding-left:0px;padding-right:0px;padding-top:10px;padding-bottom:10px;overflow-x:auto;overflow-y:auto;width:auto;height:auto;">
<style>#nogavjqqce table {
  font-family: system-ui, 'Segoe UI', Roboto, Helvetica, Arial, sans-serif, 'Apple Color Emoji', 'Segoe UI Emoji', 'Segoe UI Symbol', 'Noto Color Emoji';
  -webkit-font-smoothing: antialiased;
  -moz-osx-font-smoothing: grayscale;
}
&#10;#nogavjqqce thead, #nogavjqqce tbody, #nogavjqqce tfoot, #nogavjqqce tr, #nogavjqqce td, #nogavjqqce th {
  border-style: none;
}
&#10;#nogavjqqce p {
  margin: 0;
  padding: 0;
}
&#10;#nogavjqqce .gt_table {
  display: table;
  border-collapse: collapse;
  line-height: normal;
  margin-left: auto;
  margin-right: auto;
  color: #333333;
  font-size: 16px;
  font-weight: normal;
  font-style: normal;
  background-color: #FFFFFF;
  width: 80%;
  border-top-style: solid;
  border-top-width: 2px;
  border-top-color: #A8A8A8;
  border-right-style: none;
  border-right-width: 2px;
  border-right-color: #D3D3D3;
  border-bottom-style: solid;
  border-bottom-width: 2px;
  border-bottom-color: #A8A8A8;
  border-left-style: none;
  border-left-width: 2px;
  border-left-color: #D3D3D3;
}
&#10;#nogavjqqce .gt_caption {
  padding-top: 4px;
  padding-bottom: 4px;
}
&#10;#nogavjqqce .gt_title {
  color: #333333;
  font-size: 125%;
  font-weight: initial;
  padding-top: 4px;
  padding-bottom: 4px;
  padding-left: 5px;
  padding-right: 5px;
  border-bottom-color: #FFFFFF;
  border-bottom-width: 0;
}
&#10;#nogavjqqce .gt_subtitle {
  color: #333333;
  font-size: 85%;
  font-weight: initial;
  padding-top: 3px;
  padding-bottom: 5px;
  padding-left: 5px;
  padding-right: 5px;
  border-top-color: #FFFFFF;
  border-top-width: 0;
}
&#10;#nogavjqqce .gt_heading {
  background-color: #FFFFFF;
  text-align: center;
  border-bottom-color: #FFFFFF;
  border-left-style: none;
  border-left-width: 1px;
  border-left-color: #D3D3D3;
  border-right-style: none;
  border-right-width: 1px;
  border-right-color: #D3D3D3;
}
&#10;#nogavjqqce .gt_bottom_border {
  border-bottom-style: solid;
  border-bottom-width: 2px;
  border-bottom-color: #D3D3D3;
}
&#10;#nogavjqqce .gt_col_headings {
  border-top-style: solid;
  border-top-width: 2px;
  border-top-color: #D3D3D3;
  border-bottom-style: solid;
  border-bottom-width: 2px;
  border-bottom-color: #D3D3D3;
  border-left-style: none;
  border-left-width: 1px;
  border-left-color: #D3D3D3;
  border-right-style: none;
  border-right-width: 1px;
  border-right-color: #D3D3D3;
}
&#10;#nogavjqqce .gt_col_heading {
  color: #333333;
  background-color: #FFFFFF;
  font-size: 100%;
  font-weight: normal;
  text-transform: inherit;
  border-left-style: none;
  border-left-width: 1px;
  border-left-color: #D3D3D3;
  border-right-style: none;
  border-right-width: 1px;
  border-right-color: #D3D3D3;
  vertical-align: bottom;
  padding-top: 5px;
  padding-bottom: 6px;
  padding-left: 5px;
  padding-right: 5px;
  overflow-x: hidden;
}
&#10;#nogavjqqce .gt_column_spanner_outer {
  color: #333333;
  background-color: #FFFFFF;
  font-size: 100%;
  font-weight: normal;
  text-transform: inherit;
  padding-top: 0;
  padding-bottom: 0;
  padding-left: 4px;
  padding-right: 4px;
}
&#10;#nogavjqqce .gt_column_spanner_outer:first-child {
  padding-left: 0;
}
&#10;#nogavjqqce .gt_column_spanner_outer:last-child {
  padding-right: 0;
}
&#10;#nogavjqqce .gt_column_spanner {
  border-bottom-style: solid;
  border-bottom-width: 2px;
  border-bottom-color: #D3D3D3;
  vertical-align: bottom;
  padding-top: 5px;
  padding-bottom: 5px;
  overflow-x: hidden;
  display: inline-block;
  width: 100%;
}
&#10;#nogavjqqce .gt_spanner_row {
  border-bottom-style: hidden;
}
&#10;#nogavjqqce .gt_group_heading {
  padding-top: 8px;
  padding-bottom: 8px;
  padding-left: 5px;
  padding-right: 5px;
  color: #333333;
  background-color: #FFFFFF;
  font-size: 100%;
  font-weight: initial;
  text-transform: inherit;
  border-top-style: solid;
  border-top-width: 2px;
  border-top-color: #D3D3D3;
  border-bottom-style: solid;
  border-bottom-width: 2px;
  border-bottom-color: #D3D3D3;
  border-left-style: none;
  border-left-width: 1px;
  border-left-color: #D3D3D3;
  border-right-style: none;
  border-right-width: 1px;
  border-right-color: #D3D3D3;
  vertical-align: middle;
  text-align: left;
}
&#10;#nogavjqqce .gt_empty_group_heading {
  padding: 0.5px;
  color: #333333;
  background-color: #FFFFFF;
  font-size: 100%;
  font-weight: initial;
  border-top-style: solid;
  border-top-width: 2px;
  border-top-color: #D3D3D3;
  border-bottom-style: solid;
  border-bottom-width: 2px;
  border-bottom-color: #D3D3D3;
  vertical-align: middle;
}
&#10;#nogavjqqce .gt_from_md > :first-child {
  margin-top: 0;
}
&#10;#nogavjqqce .gt_from_md > :last-child {
  margin-bottom: 0;
}
&#10;#nogavjqqce .gt_row {
  padding-top: 8px;
  padding-bottom: 8px;
  padding-left: 5px;
  padding-right: 5px;
  margin: 10px;
  border-top-style: solid;
  border-top-width: 1px;
  border-top-color: #D3D3D3;
  border-left-style: none;
  border-left-width: 1px;
  border-left-color: #D3D3D3;
  border-right-style: none;
  border-right-width: 1px;
  border-right-color: #D3D3D3;
  vertical-align: middle;
  overflow-x: hidden;
}
&#10;#nogavjqqce .gt_stub {
  color: #333333;
  background-color: #FFFFFF;
  font-size: 100%;
  font-weight: initial;
  text-transform: inherit;
  border-right-style: solid;
  border-right-width: 2px;
  border-right-color: #D3D3D3;
  padding-left: 5px;
  padding-right: 5px;
}
&#10;#nogavjqqce .gt_stub_row_group {
  color: #333333;
  background-color: #FFFFFF;
  font-size: 100%;
  font-weight: initial;
  text-transform: inherit;
  border-right-style: solid;
  border-right-width: 2px;
  border-right-color: #D3D3D3;
  padding-left: 5px;
  padding-right: 5px;
  vertical-align: top;
}
&#10;#nogavjqqce .gt_row_group_first td {
  border-top-width: 2px;
}
&#10;#nogavjqqce .gt_row_group_first th {
  border-top-width: 2px;
}
&#10;#nogavjqqce .gt_summary_row {
  color: #333333;
  background-color: #FFFFFF;
  text-transform: inherit;
  padding-top: 8px;
  padding-bottom: 8px;
  padding-left: 5px;
  padding-right: 5px;
}
&#10;#nogavjqqce .gt_first_summary_row {
  border-top-style: solid;
  border-top-color: #D3D3D3;
}
&#10;#nogavjqqce .gt_first_summary_row.thick {
  border-top-width: 2px;
}
&#10;#nogavjqqce .gt_last_summary_row {
  padding-top: 8px;
  padding-bottom: 8px;
  padding-left: 5px;
  padding-right: 5px;
  border-bottom-style: solid;
  border-bottom-width: 2px;
  border-bottom-color: #D3D3D3;
}
&#10;#nogavjqqce .gt_grand_summary_row {
  color: #333333;
  background-color: #FFFFFF;
  text-transform: inherit;
  padding-top: 8px;
  padding-bottom: 8px;
  padding-left: 5px;
  padding-right: 5px;
}
&#10;#nogavjqqce .gt_first_grand_summary_row {
  padding-top: 8px;
  padding-bottom: 8px;
  padding-left: 5px;
  padding-right: 5px;
  border-top-style: double;
  border-top-width: 6px;
  border-top-color: #D3D3D3;
}
&#10;#nogavjqqce .gt_last_grand_summary_row_top {
  padding-top: 8px;
  padding-bottom: 8px;
  padding-left: 5px;
  padding-right: 5px;
  border-bottom-style: double;
  border-bottom-width: 6px;
  border-bottom-color: #D3D3D3;
}
&#10;#nogavjqqce .gt_striped {
  background-color: rgba(128, 128, 128, 0.05);
}
&#10;#nogavjqqce .gt_table_body {
  border-top-style: solid;
  border-top-width: 2px;
  border-top-color: #D3D3D3;
  border-bottom-style: solid;
  border-bottom-width: 2px;
  border-bottom-color: #D3D3D3;
}
&#10;#nogavjqqce .gt_footnotes {
  color: #333333;
  background-color: #FFFFFF;
  border-bottom-style: none;
  border-bottom-width: 2px;
  border-bottom-color: #D3D3D3;
  border-left-style: none;
  border-left-width: 2px;
  border-left-color: #D3D3D3;
  border-right-style: none;
  border-right-width: 2px;
  border-right-color: #D3D3D3;
}
&#10;#nogavjqqce .gt_footnote {
  margin: 0px;
  font-size: 90%;
  padding-top: 4px;
  padding-bottom: 4px;
  padding-left: 5px;
  padding-right: 5px;
}
&#10;#nogavjqqce .gt_sourcenotes {
  color: #333333;
  background-color: #FFFFFF;
  border-bottom-style: none;
  border-bottom-width: 2px;
  border-bottom-color: #D3D3D3;
  border-left-style: none;
  border-left-width: 2px;
  border-left-color: #D3D3D3;
  border-right-style: none;
  border-right-width: 2px;
  border-right-color: #D3D3D3;
}
&#10;#nogavjqqce .gt_sourcenote {
  font-size: 90%;
  padding-top: 4px;
  padding-bottom: 4px;
  padding-left: 5px;
  padding-right: 5px;
}
&#10;#nogavjqqce .gt_left {
  text-align: left;
}
&#10;#nogavjqqce .gt_center {
  text-align: center;
}
&#10;#nogavjqqce .gt_right {
  text-align: right;
  font-variant-numeric: tabular-nums;
}
&#10;#nogavjqqce .gt_font_normal {
  font-weight: normal;
}
&#10;#nogavjqqce .gt_font_bold {
  font-weight: bold;
}
&#10;#nogavjqqce .gt_font_italic {
  font-style: italic;
}
&#10;#nogavjqqce .gt_super {
  font-size: 65%;
}
&#10;#nogavjqqce .gt_footnote_marks {
  font-size: 75%;
  vertical-align: 0.4em;
  position: initial;
}
&#10;#nogavjqqce .gt_asterisk {
  font-size: 100%;
  vertical-align: 0;
}
&#10;#nogavjqqce .gt_indent_1 {
  text-indent: 5px;
}
&#10;#nogavjqqce .gt_indent_2 {
  text-indent: 10px;
}
&#10;#nogavjqqce .gt_indent_3 {
  text-indent: 15px;
}
&#10;#nogavjqqce .gt_indent_4 {
  text-indent: 20px;
}
&#10;#nogavjqqce .gt_indent_5 {
  text-indent: 25px;
}
&#10;#nogavjqqce .katex-display {
  display: inline-flex !important;
  margin-bottom: 0.75em !important;
}
&#10;#nogavjqqce div.Reactable > div.rt-table > div.rt-thead > div.rt-tr.rt-tr-group-header > div.rt-th-group:after {
  height: 0px !important;
}
</style>
<table class="gt_table" data-quarto-disable-processing="false" data-quarto-bootstrap="false">
  <thead>
    <tr class="gt_col_headings">
      <th class="gt_col_heading gt_columns_bottom_border gt_right" rowspan="1" colspan="1" scope="col" id="r"><span data-qmd-base64="JHIk"><span class='gt_from_md'>\(r\)</span></span></th>
      <th class="gt_col_heading gt_columns_bottom_border gt_right" rowspan="1" colspan="1" scope="col" id="y1"><span data-qmd-base64="JHlfMSQ="><span class='gt_from_md'>\(y_1\)</span></span></th>
      <th class="gt_col_heading gt_columns_bottom_border gt_right" rowspan="1" colspan="1" scope="col" id="y2"><span data-qmd-base64="JHlfMiQ="><span class='gt_from_md'>\(y_2\)</span></span></th>
      <th class="gt_col_heading gt_columns_bottom_border gt_right" rowspan="1" colspan="1" scope="col" id="y3"><span data-qmd-base64="JHlfMyQ="><span class='gt_from_md'>\(y_3\)</span></span></th>
      <th class="gt_col_heading gt_columns_bottom_border gt_right" rowspan="1" colspan="1" scope="col" id="pattern">Pattern</th>
    </tr>
  </thead>
  <tbody class="gt_table_body">
    <tr><td headers="r" class="gt_row gt_right">10</td>
<td headers="y1" class="gt_row gt_right">2</td>
<td headers="y2" class="gt_row gt_right">1</td>
<td headers="y3" class="gt_row gt_right">1</td>
<td headers="pattern" class="gt_row gt_right">211</td></tr>
    <tr><td headers="r" class="gt_row gt_right">11</td>
<td headers="y1" class="gt_row gt_right">2</td>
<td headers="y2" class="gt_row gt_right">1</td>
<td headers="y3" class="gt_row gt_right">2</td>
<td headers="pattern" class="gt_row gt_right">212</td></tr>
    <tr><td headers="r" class="gt_row gt_right">12</td>
<td headers="y1" class="gt_row gt_right">2</td>
<td headers="y2" class="gt_row gt_right">1</td>
<td headers="y3" class="gt_row gt_right">3</td>
<td headers="pattern" class="gt_row gt_right">213</td></tr>
    <tr><td headers="r" class="gt_row gt_right">13</td>
<td headers="y1" class="gt_row gt_right">2</td>
<td headers="y2" class="gt_row gt_right">2</td>
<td headers="y3" class="gt_row gt_right">1</td>
<td headers="pattern" class="gt_row gt_right">221</td></tr>
    <tr><td headers="r" class="gt_row gt_right">14</td>
<td headers="y1" class="gt_row gt_right">2</td>
<td headers="y2" class="gt_row gt_right">2</td>
<td headers="y3" class="gt_row gt_right">2</td>
<td headers="pattern" class="gt_row gt_right">222</td></tr>
    <tr><td headers="r" class="gt_row gt_right">15</td>
<td headers="y1" class="gt_row gt_right">2</td>
<td headers="y2" class="gt_row gt_right">2</td>
<td headers="y3" class="gt_row gt_right">3</td>
<td headers="pattern" class="gt_row gt_right">223</td></tr>
    <tr><td headers="r" class="gt_row gt_right">16</td>
<td headers="y1" class="gt_row gt_right">2</td>
<td headers="y2" class="gt_row gt_right">3</td>
<td headers="y3" class="gt_row gt_right">1</td>
<td headers="pattern" class="gt_row gt_right">231</td></tr>
    <tr><td headers="r" class="gt_row gt_right">17</td>
<td headers="y1" class="gt_row gt_right">2</td>
<td headers="y2" class="gt_row gt_right">3</td>
<td headers="y3" class="gt_row gt_right">2</td>
<td headers="pattern" class="gt_row gt_right">232</td></tr>
    <tr><td headers="r" class="gt_row gt_right">18</td>
<td headers="y1" class="gt_row gt_right">2</td>
<td headers="y2" class="gt_row gt_right">3</td>
<td headers="y3" class="gt_row gt_right">3</td>
<td headers="pattern" class="gt_row gt_right">233</td></tr>
  </tbody>
  &#10;  
</table>
</div>
</div>
</div></td>
</tr>
</tbody>
</table>

Table 1: Response patterns for $p=3$ with $m_1=2$, and $m_2=m_3=3$.

Later on we wish to use lower-order residuals to assess the fit of a model to the data, which first requires a description of lower-order moments and its connection to the joint response probabilities. <!-- Let $I_{ik} = [y_i = k]$ be the indicator variable for the event that $y_i$ takes the value $k$, where $[\cdot]$ is the Iverson bracket. --> Marginally, each $y_i$ can be viewed as a multinoulli random variable with event probabilities $\pi^{(i)}_k = \Pr(y_i = k)$, $k=1,\dots m_i$, that sum to one. Therefore, this univariate distribution is characterised by its $(m_i-1)$ *moments* $\pi^{(i)}_2,\dots,\pi^{(i)}_{m_i}$, with the first moment being redundant due to the sum to unity constraint. All univariate moments can be collected into the vector $\dot{\boldsymbol\pi}_1 = (\pi^{(i)}_k)^\top$ whose dimension is $S_1 = \sum_i (m_i-1)$. In a similar light, the bivariate distribution of $(y_i, y_j)$ is characterised by its $(m_i-1)(m_j-1)$ *joint moments* $\pi^{(ij)}_{k,l} = \Pr(y_i = k, y_j = l)$, $k=2,\dots,m_i$, $l=2,\dots,m_j$. Also collect all bivariate moments into the vector $\dot{\boldsymbol\pi}_2 = (\pi^{(ij)}_{k,l})^\top$ whose dimension is $S_2 = \sum_{i<j} (m_i-1)(m_j-1)$. Finally, denote by $\boldsymbol\pi_2 = (\dot{\boldsymbol\pi}_1^\top, \dot{\boldsymbol\pi}_2^\top)^\top$ the vector of multivariate moments up to order 2, which is a vector of length $S = S_1 + S_2$.

Because the lower order moments are contained in the higher order moments, the vector $\boldsymbol\pi_2$ can be extracted from the joint probabilities ${\boldsymbol\pi}$ via a linear operation ${\boldsymbol\pi}_2 = {\mathbf T}_2 {\boldsymbol\pi}$ ([Jamil et al., 2025](#ref-jamil2025pairwise)). As an example, continuing from the $p=3$ instance above, the moments for the first variable $y_1$, $\Pr(y_1=2)$ can be obtained by *summing* over all joint probabilities whose patterns contain $y_1=2$. The positions of these joint probabilities in the vector ${\boldsymbol\pi}$ are picked up by the first row of the matrix ${\mathbf T}_2$. Similarly, the two bivariate moments of $(y_1,y_2)$, i.e. $\pi^{(12)}_{22}$ and $\pi^{(12)}_{23}$ are obtained by summing over the joint probabilities whose patterns contain $y_1=2$ and $y_2=2$, and $y_1=2$ and $y_2=3$, respectively.

``` r
options(width = 100)
create_T2_mat <- function(m) {
  # m: integer vector of length p, where m[i] = number of categories of variable i
  p <- length(m)
  # 1) all joint patterns (rows = ∏ m[i], cols = p)
  patterns <- expand.grid(rev(lapply(m, seq_len)), KEEP.OUT.ATTRS = FALSE, stringsAsFactors = FALSE)
  patterns <- patterns[, rev(seq_len(p))] # reverse to match y1, y2, ...
  n_pat <- nrow(patterns)
  
  # 2) precompute total number of rows: sum_i (m[i]-1) + sum_{i<j} (m[i]-1)*(m[j]-1)
  uni_rows <- sum(m - 1)
  biv_rows <- 0L
  for(i in seq_len(p-1)) for(j in (i+1):p)
    biv_rows <- biv_rows + (m[i]-1)*(m[j]-1)
  total_rows <- uni_rows + biv_rows
  
  # 3) allocate
  out <- matrix(0L, nrow = total_rows, ncol = n_pat)
  rn  <- character(total_rows)
  
  # 4) fill univariate indicator rows
  r <- 1L
  for(i in seq_len(p)) {
    for(cat in 2:m[i]) {
      out[r, ] <- as.integer(patterns[[i]] == cat)
      rn[r]   <- paste0("Y", i, "=", cat)
      r       <- r + 1L
    }
  }
  
  # 5) fill bivariate indicator rows
  for(i in seq_len(p-1)) for(j in (i+1):p) {
    for(c1 in 2:m[i]) for(c2 in 2:m[j]) {
      out[r, ] <- as.integer(patterns[[i]] == c1 & patterns[[j]] == c2)
      rn[r]   <- paste0("Y", i, "=", c1, ",Y", j, "=", c2)
      r       <- r + 1L
    }
  }
  
  rownames(out) <- rn
  colnames(out) <- apply(patterns, 1, paste0, collapse = "")
  out
}
create_T2_mat(c(2, 3, 3))
```

              111 112 113 121 122 123 131 132 133 211 212 213 221 222 223 231 232 233
    Y1=2        0   0   0   0   0   0   0   0   0   1   1   1   1   1   1   1   1   1
    Y2=2        0   0   0   1   1   1   0   0   0   0   0   0   1   1   1   0   0   0
    Y2=3        0   0   0   0   0   0   1   1   1   0   0   0   0   0   0   1   1   1
    Y3=2        0   1   0   0   1   0   0   1   0   0   1   0   0   1   0   0   1   0
    Y3=3        0   0   1   0   0   1   0   0   1   0   0   1   0   0   1   0   0   1
    Y1=2,Y2=2   0   0   0   0   0   0   0   0   0   0   0   0   1   1   1   0   0   0
    Y1=2,Y2=3   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   1   1   1
    Y1=2,Y3=2   0   0   0   0   0   0   0   0   0   0   1   0   0   1   0   0   1   0
    Y1=2,Y3=3   0   0   0   0   0   0   0   0   0   0   0   1   0   0   1   0   0   1
    Y2=2,Y3=2   0   0   0   0   1   0   0   0   0   0   0   0   0   1   0   0   0   0
    Y2=2,Y3=3   0   0   0   0   0   1   0   0   0   0   0   0   0   0   1   0   0   0
    Y2=3,Y3=2   0   0   0   0   0   0   0   1   0   0   0   0   0   0   0   0   1   0
    Y2=3,Y3=3   0   0   0   0   0   0   0   0   1   0   0   0   0   0   0   0   0   1

Figure 1: Matrix ${\mathbf T}_2$ for the case of $p=3$ with $m_1=2$, and $m_2=m_3=3$.

Note that this construction of lower-order moments generalises to any order $q \le p$, but the total number of moments up to order $q$ grows combinatorially in both $p$ and the category counts $m_i$, yielding design matrices $\mathbf{T}_q$ that can become computationally burdensome. Moreover, although we arbitrarily dropped the first moment in the foregoing construction, the choice of which category to omit is immaterial. This is because category probabilities sum to one, so excluding any one category produces a similar-dimensional parameterisation algebraically equivalent to excluding any other. For further details, consult Reiser ([1996](#ref-reiser1996analysis)) and Maydeu-Olivares & Joe ([2006](#ref-maydeu2006limited)).

### 2.2 Confirmatory factor analysis

### 2.3 Parameter estimation

### 2.4 Distribution of residuals

### 2.5 Wald type tests

### 2.6 Pearson type tests

### 2.7 General GOF tests

### 2.8 Estimation of degrees of freedom

## 3 Usage

## References

Jamil, H., Moustaki, I., & Skinner, C. (2025). Pairwise likelihood estimation and limited-information goodness-of-fit test statistics for binary factor analysis models under complex survey sampling. *British Journal of Mathematical and Statistical Psychology*, *78*(1), 258–285. <https://doi.org/10.1111/bmsp.12358>

Maydeu-Olivares, A., & Joe, H. (2006). Limited information goodness-of-fit testing in multidimensional contingency tables. *Psychometrika*, *71*(4), 713.

Reiser, M. (1996). Analysis of residuals for the multionmial item response model. *Psychometrika*, *61*(3), 509–528. <https://doi.org/10.1007/BF02294552>